In [7]:
from grafo import Grafo
import pandas as pd
import math

In [2]:
cruces = pd.read_csv("data/cruces_clean.csv")
calles = pd.read_csv("data/direcciones_clean.csv")
g = Grafo()

In [53]:
cruces

,id_via,nombre_via,id_via_cruzada,nombre_via_cruzada,x,y
0,127,CALLE DE ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA,44260746,448479753
1,127,CALLE DE ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU,44260732,448479748
2,127,CALLE DE ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU,44290433,448494484
3,127,CALLE DE ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE,44320997,448510385
4,127,CALLE DE ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN,44322285,448530840
...,...,...,...,...,...,...
36160,31003706,CALLE DE TREN OBRERO,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ,44083665,446665463
36161,31004081,GLORIETA DE CIRILO MARTÍN MARTÍN,369600,CARRETERA DE HÚMERA,43390886,447883842
36162,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,29900,CALLE DE ALICANTE,44145890,447199432
36163,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,2729,CALLE DEL GRANITO,44150370,447189024


In [55]:
for _, cruce in cruces.drop_duplicates(["x", "y"]).iterrows():
    g.agregar_vertice((cruce["x"], cruce["y"]))

In [67]:
def add_arista(df: pd.DataFrame):
    cruce1 = (df.iloc[0]["x"], df.iloc[0]["y"])
    for _, calle in pd.concat([df["nombre_via"], df["nombre_via_cruzada"]]).drop_duplicates().items():
        for _, cruce in cruces[(cruces["nombre_via"] == calle)|(cruces["nombre_via_cruzada"] == calle)].iterrows():
            g.agregar_arista(cruce1, (cruce["x"], cruce["y"]), (calle), 1) # poner pesos

In [68]:
cruces.groupby(["x", "y"]).apply(add_arista)
g.aristas

{((42912532, 448079541),
  (42984570, 448069175)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42984570, 448069175),
  (42912532, 448079541)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42912532, 448079541),
  (42948986, 448073915)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42948986, 448073915),
  (42912532, 448079541)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42912532, 448079541),
  (42942293, 448074791)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42942293, 448074791),
  (42912532, 448079541)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42912532, 448079541),
  (42912532, 448079541)): {'data': 'AVENIDA DE LA ESTACIÓN', 'weight': 1},
 ((42912532, 448079541),
  (42925621, 448077155)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42925621, 448077155),
  (42912532, 448079541)): {'data': 'CALLE DE CUEVAS DEL VALLE', 'weight': 1},
 ((42912532, 448079541),
  (42934544, 448075812)): {'data': 'CALLE DE CUEVAS

In [3]:
calles

,Direccion completa,Codigo de via,x,y
0,AUTOV A-1 1000 EN,31001337,44305633,448250340
1,AUTOV A-1 1000 SA,31001337,44312246,448249077
2,AUTOV A-1 1100 EN,31001337,44367522,448330933
3,AUTOV A-1 1100 SA,31001337,44373755,448325151
4,AUTOV A-1 1200 EN,31001337,44425207,448408185
...,...,...,...,...
210352,PASEO ZURRÓN 49,725,45051964,448025888
210353,PASEO ZURRÓN 51,725,45058080,448025960
210354,PASEO ZURRÓN 53,725,45061025,448027575
210355,PASEO ZURRÓN 55,725,45063500,448027080


In [4]:
calles.drop_duplicates("Codigo de via")

,Direccion completa,Codigo de via,x,y
0,AUTOV A-1 1000 EN,31001337,44305633,448250340
8,AUTOV A-2 500 EN,31001336,44469993,447782563
37,AUTOV A-3 400 EN,31001342,44439783,447321577
77,AUTOV A-4 400 EN,31001334,44167422,447063954
96,AUTOV A-42 500 EN,31001341,43905717,447000585
...,...,...,...,...
210105,ZURBANO 1,807000,44124688,447560119
210206,ZURBARÁN 1,807100,44106724,447590046
210233,ZURICH 1,2462,44840460,447486805
210258,ZURITA 1,807300,44088069,447375916


In [16]:
def asociar(row):
    df = calles[calles["Direccion completa"].str.contains(row["nombre_via"])]
    if df.empty:
        return None
    df["distance"] = df.apply(lambda row2: (row["x"]-row2["x"])**2+(row["y"]-row2["y"])**2, axis=1)
    return df.loc[df["distance"].idxmax()]["Direccion completa"]

In [ ]:
cruces["direccion_asociada"] = cruces.apply(asociar, axis=1)
cruces